## TP05: Fenómeno de El Niño Oscilación Sur (ENSO) - Impacto de la Niña (2020-202)

En el siguiente sitio web encontrará información sobre el ENSO y como se utiliza la
temperatura superficial del mar (SST) para su monitoreo.
https://www.ncei.noaa.gov/access/monitoring/enso/sst3

In [ ]:
import os

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as  np

import smn #  libreria propia

import matplotlib.axes as maxes
import matplotlib.pyplot as plt
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.crs import PlateCarree
from cartopy.feature import LAND, COASTLINE, RIVERS
from cmocean.cm import thermal, amp, rain
from mpl_toolkits.axes_grid1 import make_axes_locatable

import netCDF4 as nc

DATA_DIR = "./data"

In [ ]:
def between(v, vmin, vmax):
    return (v > vmin) and (v < vmax)

def anomalia(v, pos=0.5, neg=-0.5):
    if between(v, pos, np.inf):
        return "pos"
    elif between(v, -np.inf, neg):
        return "neg"
    else:
        return "neutro"
    
def extract_from_region(var, lat, lon, region):
    """
    extract values of a variable betweeen lat,lon coordenates.

    """
    def _order_minmax(point):
        if point[1] > point[0]:
            return point[0], point[1]
        else:
            return point[1], point[0]

    def get_index_limits(vardim, limits):
        return [np.abs(vardim - limit).argmin() for limit in limits]

    def reduce_dim(vardim, idxs):
        m, M =  _order_minmax(idxs)
        return vardim[m:M]

    def extract_var(var, lat_idxs, lon_idxs):
        lat_m, lat_M =  _order_minmax(lat_idxs)
        lon_m, lon_M =  _order_minmax(lon_idxs)
        return var[lat_m : lat_M, lon_m : lon_M]

    lat_idxs = get_index_limits(lat, region["lat"])
    lon_idxs = get_index_limits(lon, region["lon"])
    if type(var) == dict:
        _var = dict()
        for nvar, vvar in var.items():
            _var[nvar] = extract_var(vvar, lat_idxs, lon_idxs)
    else:
        _var = extract_var(var, lat_idxs, lon_idxs)

    return _var, reduce_dim(lat, lat_idxs), reduce_dim(lon, lon_idxs)

**1. En función de la serie de ONI (Oceanic Niño Index) de la región El Niño 3.4 (5°S-5°N, 120°O-170°O), elegir tres meses que representen la última Niña (fase negativa entre 2020 y 2023), el estado neutro previo (anomalías cercanas a cero) y el último Niño (fase positiva entre 2018 y 2020). Para cada caso  promediar los tres meses elegidos y graficar el mapa de una región que abarque el Pacífico Ecuatorial y parte de la Plataforma Argentpor ejemplo, 180ºO-45ºO, 45ºS-20ºN) con la herramienta online GIOVANNI**

In [ ]:
nf_oni = "oni.csv"
nf_sst =  "sst.csv"

In [ ]:
dir_oni = os.path.join(DATA_DIR, nf_oni)
if os.path.exists(dir_oni):
    oni = pd.read_csv(dir_oni)
else: 
    oni_url = "https://www.cpc.ncep.noaa.gov/data/indices/oni.ascii.txt"
    oni = smn.get_smn_data(oni_url, var="oni")
    oni.to_csv(dir_oni, index=False)

In [ ]:
dir_sst = os.path.join(DATA_DIR, nf_sst)
if os.path.exists(dir_sst):
    sst = pd.read_csv(dir_sst)
else:
    sst_url = "https://www.cpc.ncep.noaa.gov/data/indices/sstoi.indices"
    sst = smn.get_smn_data(sst_url, var="sst")
    sst.to_csv(dir_sst, index=False)

In [ ]:
oni["estado"] = oni.anom.apply(lambda x: anomalia(x, pos=.25, neg=-0.25))
oni["label"] = oni.apply(lambda row: "{}_{}".format(row.year, row.seas), axis=1)
oni = oni[oni.year > 2017].copy()
oni.reset_index(inplace=True)
sst = sst[sst.year>2017].copy()
sst.reset_index(inplace=True)

In [ ]:
plt.figure(figsize=(15, 5))

plt.title("Indice del Niño", fontsize=20)
plt.plot(oni.anom, "--", color="dimgrey")

plt.scatter(5, -0.01, s=200, marker="*", label="Anomalía nutra  elegido", color="green")
plt.scatter(10, 0.885, s=200, marker="*", label="Anomalía positiva elegida", color="red")
plt.scatter(34, -1.27, s=200, marker="*", label="Anomalía negativa elegida", color="blue")


plt.plot(oni[oni.estado=="pos"].anom, "o", color="crimson", alpha=0.8, label="Positivas")
plt.plot(oni[oni.estado=="neg"].anom, "o", color="darkcyan", alpha=0.8, label="Negativas")
plt.plot(oni[oni.estado=="neutro"].anom, "o", color="greenyellow", alpha=0.8, label="Neutro")

plt.fill_between([29, 61], [-1.4, -1.4], [1.1, 1.1], color="darkcyan", alpha=0.3,)
plt.fill_between([8, 26], [-1.4, -1.4], [1.1, 1.1], color="crimson", alpha=0.3,)

plt.xticks(ticks=[0, 12, 24, 36, 48, 60],  labels=[2018, 2019, 2020, 2021, 2022, 2023], rotation=90)

plt.hlines(0, 70, 0, color="red", linestyles="dashed", alpha=0.7)
plt.xlabel("Año")
plt.ylabel("Temperatura (°C)")
plt.legend()
plt.grid()
plt.show()

print("Figura 1: Indice del Niño. En rojo las anomalías positivas (niño). En celestre las anomalías negativas (niña). En verde se muestras las anomalías neutras o puntos de anomalía cero. Los puntos con forma de estrella son las anomalías elegidas para analizar en detalle.")

In [ ]:
# Octubre-Noviembre-Diciembre 2020 -  anomin
# Octubre-Noviembre-Diciembre 2018 -  anomax
# Mayo-Junio-Julio 2018 - Neutrolidad

In [ ]:
region = dict(
    lon=[-180, -45],
    lat=[-30, 20]
)

region_34 = dict(
    lon=[-170, -120],
    lat=[-5, 5]
)


In [ ]:
nf_neutro = "neutro_sst_modis_monthly_4km.nc"
nf_anomin = "anom_min_sst_modis_monthly_4km.nc"
nf_anomax = "anom_max_sst_modis_monthly_4km.nc"

neutro = nc.Dataset(os.path.join(DATA_DIR,  nf_neutro))
anomin = nc.Dataset(os.path.join(DATA_DIR,  nf_anomin))
anomax = nc.Dataset(os.path.join(DATA_DIR,  nf_anomax))

In [ ]:
sst_neutro = neutro.variables["MODISA_L3m_SST4_Monthly_4km_R2019_0_sst4"][:]
lat_neutro = neutro.variables["lat"][:]
lon_neutro = neutro.variables["lon"][:]

sst_anomin = anomin.variables["MODISA_L3m_SST4_Monthly_4km_R2019_0_sst4"][:]
lat_anomin = anomin.variables["lat"][:]
lon_anomin = anomin.variables["lon"][:]

sst_anomax = anomax.variables["MODISA_L3m_SST4_Monthly_4km_R2019_0_sst4"][:]
lat_anomax = anomax.variables["lat"][:]
lon_anomax = anomax.variables["lon"][:]

sst_neutro, lat_neutro, lon_neutro  = extract_from_region(sst_neutro, lat_neutro, lon_neutro, region=region)
sst_anomin, lat_anomin, lon_anomin  = extract_from_region(sst_anomin, lat_anomin, lon_anomin, region=region)
sst_anomax, lat_anomax, lon_anomax  = extract_from_region(sst_anomax, lat_anomax, lon_anomax, region=region)

In [ ]:
levels = np.arange(21, 31, 1)

In [ ]:
fig, axs = plt.subplots(3, 1, subplot_kw={'projection': PlateCarree()}, figsize=(10, 10), sharex=True, sharey=True)


axs[0].set_title("SST Anomalia MAX \n(Oct-Nov-Dic 2018)", fontsize=8)
im2 = axs[0].contourf(lon_anomax,  lat_anomax, sst_anomax, vmin=21, vmax=31, levels=levels, cmap=thermal, zorder=0)
divider = make_axes_locatable(axs[0])
cax = divider.append_axes('bottom', size='5%', pad=0.4, axes_class=maxes.Axes)
cbar2 = fig.colorbar(im2, cax=cax, orientation='horizontal' )
for label in cbar2.ax.get_xticklabels():
    label.set_rotation(45)
    
axs[1].set_title("SST Neutral \n(May-Jun-Jul 2018)", fontsize=8)
im2 = axs[1].contourf(lon_neutro,  lat_neutro, sst_neutro, vmin=21, vmax=31, levels=levels, cmap=thermal, zorder=0)
divider = make_axes_locatable(axs[1])
cax = divider.append_axes('bottom', size='5%', pad=0.4, axes_class=maxes.Axes)
cbar2 = fig.colorbar(im2, cax=cax, orientation='horizontal' )
for label in cbar2.ax.get_xticklabels():
    label.set_rotation(45)

axs[2].set_title("SST Anomalia MIN \n(Oct-Nov-Dic 2020)", fontsize=8)
im2 = axs[2].contourf(lon_anomin,  lat_anomin, sst_anomin, vmin=21, vmax=31, levels=levels, cmap=thermal, zorder=0)
divider = make_axes_locatable(axs[2])
cax = divider.append_axes('bottom', size='5%', pad=0.4, axes_class=maxes.Axes)
cbar2 = fig.colorbar(im2,  cax=cax, orientation='horizontal' )
for label in cbar2.ax.get_xticklabels():
    label.set_rotation(45)

for i in range(3):
    axs[i].set_facecolor('white')
    axs[i].add_feature(LAND, facecolor="white", zorder=1)
    axs[i].add_feature(COASTLINE, linewidth=0.5)

    gl = axs[i].gridlines(crs=PlateCarree(), draw_labels=True, linestyle='--', zorder=0)
    gl.yformatter = LATITUDE_FORMATTER
    gl.xformatter = LONGITUDE_FORMATTER
    gl.right_labels = False
    gl.top_labels = False
    axs[i].plot([region_34["lon"][0], region_34["lon"][0], region_34["lon"][1], region_34["lon"][1], region_34["lon"][0]], 
                [region_34["lat"][1], region_34["lat"][0], region_34["lat"][0], region_34["lat"][1], region_34["lat"][1]],
                "r--", label="region 3.4")
    axs[i].legend()

fig.tight_layout()
plt.show()

print("Figura 2: Anomalias sobre el pacifico. Imagen superior anomalía máxima, la del medio anomalía neutra e imagen inferior anomalía mínima. En las 3, recuadrado se encuentra la región 3.4.")

**2. De manera descriptiva, observa diferencias de temperatura en los tres casos? Solo en el Pacifico? Realice un mapa de la diferencia entre SST fase negativa y SST fase positiva.**  

En la figura 2 podemos observar la SST en el caso de 3 anomalías definidas distintas (positíva, neutra y negativa). A simple vista la diferencia de temperatura se puede obsevar sobre todo en el pacífico y en particular en la región 3.4. Mientras más al oeste uno se traslada más notoria es la diferencia entre anomalía positiva y anomalái negativa. 

Si uno mira la figura 3, donde se visualizan las diferencias de SST entre anomalía positiva y anomalía negativa, se encuentra que a pesar de que las temperaturas más calidas se encuentran al eoste, la disparidad entre las anomalías positiva y negativa se observa mucho mejor al este de la región 3.4. 

In [ ]:
sst_anomin, lat_anomin, lon_anomin  = extract_from_region(sst_anomin, lat_anomin, lon_anomin, region=region_34)
sst_anomax, lat_anomax, lon_anomax  = extract_from_region(sst_anomax, lat_anomax, lon_anomax, region=region_34)

In [ ]:
sst_diff = sst_anomax - sst_anomin[:, 1:] 
levels = np.arange(0, 5, 1)

In [ ]:
sst_diff.max()

In [ ]:
fig, axs = plt.subplots(1, 1, subplot_kw={'projection': PlateCarree()},  figsize=(10, 10), sharex=True, sharey=True)

axs.set_title("SST Diferencia Anomalias", fontsize=8)
im2 = axs.contourf(lon_anomax,  lat_anomax, sst_diff, vmin=0, vmax=4, levels=levels, cmap=thermal, zorder=0)
divider = make_axes_locatable(axs)
cax = divider.append_axes('bottom', size='5%', pad=0.4, axes_class=maxes.Axes)
cbar2 = fig.colorbar(im2, cax=cax, orientation='horizontal' )
for label in cbar2.ax.get_xticklabels():
    label.set_rotation(45)
    
axs.set_facecolor('white')
axs.add_feature(LAND, facecolor="white", zorder=1)
axs.add_feature(COASTLINE, linewidth=0.5)

gl = axs.gridlines(crs=PlateCarree(), draw_labels=True, linestyle='--', zorder=0)
gl.yformatter = LATITUDE_FORMATTER
gl.xformatter = LONGITUDE_FORMATTER
gl.right_labels = False
gl.top_labels = False

plt.show()

print("Figura 3: Diferencia de anomalía psoitiva y negativa en la región 3.4.  Se discretiza la diferencia para resaltar las zonas de mayor disparidad de temperatura.")

#### ALTURA DE NIVEL DEL OCEANO y HUMEDAD


Repetir los promedios para los tres casos en la misma región con altura del mar y humedad de suelo. En el caso de altura del mar (SLA), descargue el producto SEALEVEL_GLO_PHY_L4_MY_008_047 en Marine Copernicus para la misma región que en 1. Los datos de percentil de humedad de suelo de la
misión GRACE se pueden descargar de GIOVANNI (producto GRACEDADM_CLSM025GL_7D) y elija una región que abarque Latinoamérica. La variación de la humedad de suelo se va a asociar a la precipitación. 

In [ ]:
from http.client import IncompleteRead

In [ ]:
import xarray as xr 
import oceanpy.copernicus.xarraydownload as xd
from oceanpy.credentials import COPERNICUS as creed

def get_mean_var(DS, lats, lons, times, var="adt", axis=0):
    ds = DS.sel(time=times, latitude=lats, longitude=lons)
    adt = ds.variables["adt"].values
    lat = ds.variables["latitude"].values
    lon = ds.variables["longitude"].values
    return dict(adt=np.nanmean(adt, axis), lat=lat, lon=lon)

DSI = "cmems_obs-sl_glo_phy-ssh_my_allsat-l4-duacs-0.25deg_P1D"

In [ ]:
session = xd.session_copernicus(user=creed["user"], pwd=creed["pwd"])
store = xd.get_dataset_store(session, DSI)
DS = xr.open_dataset(store)
DS = DS.drop_duplicates("time")
DS = DS.sortby("time")

In [ ]:
SSH_LAT = slice(region["lat"][0], region["lat"][1])
SSH_LON = slice(region["lon"][0], region["lon"][1])
SSH_TIME_ANOMIN = slice("2020-10-01T00:00:00.00000000", "2021-01-01T00:00:00.00000000")
SSH_TIME_ANOMAX = slice("2018-10-01T00:00:00.00000000", "2019-01-01T00:00:00.00000000")
SSH_TIME_NOANO = slice("2018-05-01T00:00:00.00000000", "2018-09-01T00:00:00.00000000")

In [ ]:
ssh_anomin = get_mean_var(DS, SSH_LAT, SSH_LON, SSH_TIME_ANOMIN)
ssh_anomax = get_mean_var(DS, SSH_LAT, SSH_LON, SSH_TIME_ANOMAX)
ssh_anono = get_mean_var(DS, SSH_LAT, SSH_LON, SSH_TIME_NOANO)

In [ ]:
fig, axs = plt.subplots(3, 1, subplot_kw={'projection': PlateCarree()}, figsize=(10, 10), sharex=True, sharey=True)


axs[0].set_title("SSH Anomalia MAX \n(Oct-Nov-Dic 2018)", fontsize=8)
im2 = axs[0].contourf(ssh_anomax["lon"], ssh_anomax["lat"], ssh_anomax["adt"], vmin=0.2, vmax=1.3, levels=np.arange(0.2, 1.4, 0.1), cmap=amp, zorder=0)
divider = make_axes_locatable(axs[0])
cax = divider.append_axes('bottom', size='5%', pad=0.4, axes_class=maxes.Axes)
cbar2 = fig.colorbar(im2, cax=cax, orientation='horizontal' )
for label in cbar2.ax.get_xticklabels():
    label.set_rotation(45)
    
axs[1].set_title("SSH Neutral \n(May-Jun-Jul 2018)", fontsize=8)
im2 = axs[1].contourf(ssh_anono["lon"], ssh_anomax["lat"], ssh_anono["adt"], vmin=0.2, vmax=1.3, levels=np.arange(0.2, 1.4, 0.1), cmap=amp, zorder=0)
divider = make_axes_locatable(axs[1])
cax = divider.append_axes('bottom', size='5%', pad=0.4, axes_class=maxes.Axes)
cbar2 = fig.colorbar(im2, cax=cax, orientation='horizontal' )
for label in cbar2.ax.get_xticklabels():
    label.set_rotation(45)

axs[2].set_title("SSH Anomalia MIN \n(Oct-Nov-Dic 2020)", fontsize=8)
im2 = axs[2].contourf(ssh_anomin["lon"], ssh_anomax["lat"], ssh_anomin["adt"], vmin=0.2, vmax=1.3, levels=np.arange(0.2, 1.4, 0.1), cmap=amp, zorder=0)
divider = make_axes_locatable(axs[2])
cax = divider.append_axes('bottom', size='5%', pad=0.4, axes_class=maxes.Axes)
cbar2 = fig.colorbar(im2,  cax=cax, orientation='horizontal' )
for label in cbar2.ax.get_xticklabels():
    label.set_rotation(45)

for i in range(3):
    axs[i].set_facecolor('white')
    axs[i].add_feature(LAND, facecolor="white", zorder=1)
    axs[i].add_feature(COASTLINE, linewidth=0.5)

    gl = axs[i].gridlines(crs=PlateCarree(), draw_labels=True, linestyle='--', zorder=0)
    gl.yformatter = LATITUDE_FORMATTER
    gl.xformatter = LONGITUDE_FORMATTER
    gl.right_labels = False
    gl.top_labels = False
    
    axs[i].plot([region_34["lon"][0], region_34["lon"][0], region_34["lon"][1], region_34["lon"][1], region_34["lon"][0]], 
                [region_34["lat"][1], region_34["lat"][0], region_34["lat"][0], region_34["lat"][1], region_34["lat"][1]],
                "w--", label="region 3.4")
    axs[i].legend()

fig.tight_layout()
plt.show()

print("Figura 4: Altura del nivel del mar promedio de los 3 meses de anomalía. En la imagen superior, anomalía positiva, imagen del medio, anomalía nuetra y la imagen inferior se muestra una anomaía negativa")

In [ ]:
fig, axs = plt.subplots(1, 1, subplot_kw={'projection': PlateCarree()},  figsize=(10, 10), sharex=True, sharey=True)

axs.set_title("SSH Diferencia Anomalias", fontsize=8)
im2 = axs.contourf(ssh_anomin["lon"], ssh_anomax["lat"], ssh_anomax["adt"] - ssh_anomin["adt"], cmap=amp, zorder=0)
divider = make_axes_locatable(axs)
cax = divider.append_axes('bottom', size='5%', pad=0.4, axes_class=maxes.Axes)
cbar2 = fig.colorbar(im2, cax=cax, orientation='horizontal' )
for label in cbar2.ax.get_xticklabels():
    label.set_rotation(45)
    
axs.set_facecolor('white')
axs.add_feature(LAND, facecolor="white", zorder=1)
axs.add_feature(COASTLINE, linewidth=0.5)

gl = axs.gridlines(crs=PlateCarree(), draw_labels=True, linestyle='--', zorder=0)
gl.yformatter = LATITUDE_FORMATTER
gl.xformatter = LONGITUDE_FORMATTER
gl.right_labels = False
gl.top_labels = False

axs.plot([region_34["lon"][0], region_34["lon"][0], region_34["lon"][1], region_34["lon"][1], region_34["lon"][0]], 
                [region_34["lat"][1], region_34["lat"][0], region_34["lat"][0], region_34["lat"][1], region_34["lat"][1]],
                "w--", label="region 3.4")
axs.legend()
plt.show()

print("Figura 5: Diferencias de la altura del nivel del mar entre periodos de anomalías máxima y mínima.")

In [ ]:
hum_nf_neutro = "humedad_anomalia_neutra.nc"
hum_nf_anomin = "humedad_anomalia_neg.nc"
hum_nf_anomax = "humedad_anomalia_pos.nc"

hum_neutro = nc.Dataset(os.path.join(DATA_DIR,  hum_nf_neutro))
hum_anomin = nc.Dataset(os.path.join(DATA_DIR,  hum_nf_anomin))
hum_anomax = nc.Dataset(os.path.join(DATA_DIR,  hum_nf_anomax))

In [ ]:
hum_neutro.variables.keys()

In [ ]:
regionh = {'lon': [-100, -30], 'lat': [-30, 20]}

In [ ]:
humedad_neutro = hum_neutro.variables["GRACEDADM_CLSM025GL_7D_3_0_sfsm_inst"][:]
hum_lat_neutro = hum_neutro.variables["lat"][:]
hum_lon_neutro = hum_neutro.variables["lon"][:]

humedad_anomin = hum_anomin.variables["GRACEDADM_CLSM025GL_7D_3_0_sfsm_inst"][:]
hum_lat_anomin = hum_anomin.variables["lat"][:]
hum_lon_anomin = hum_anomin.variables["lon"][:]

humedad_anomax = hum_anomax.variables["GRACEDADM_CLSM025GL_7D_3_0_sfsm_inst"][:]
hum_lat_anomax = hum_anomax.variables["lat"][:]
hum_lon_anomax = hum_anomax.variables["lon"][:]

humedad_neutro, hum_lat_neutro, hum_lon_neutro  = extract_from_region(humedad_neutro, hum_lat_neutro, hum_lon_neutro, region=regionh)
humedad_anomin, hum_lat_anomin, hum_lon_anomin  = extract_from_region(humedad_anomin, hum_lat_anomin, hum_lon_anomin, region=regionh)
humedad_anomax, hum_lat_anomax, hum_lon_anomax  = extract_from_region(humedad_anomax, hum_lat_anomax, hum_lon_anomax, region=regionh)

In [ ]:
hm = 0 #min(humedad_anomax.min(), humedad_anomin.min(), humedad_neutro.min())

In [ ]:
hM = 100 # max(humedad_anomax.max(), humedad_anomin.max(), humedad_neutro.max())

In [ ]:
hlevels = np.arange(hm, hM + 5, 5)

In [ ]:
fig, axs = plt.subplots(3, 1, subplot_kw={'projection': PlateCarree()}, figsize=(10, 10), sharex=True, sharey=True)


axs[0].set_title("SST Anomalia MAX \n(Oct-Nov-Dic 2018)", fontsize=8)
im2 = axs[0].contourf(hum_lon_anomax,  hum_lat_anomax, humedad_anomax, vmin=hm, vmax=hM, levels=hlevels, cmap=rain, zorder=0)
divider = make_axes_locatable(axs[0])
cax = divider.append_axes('bottom', size='5%', pad=0.4, axes_class=maxes.Axes)
cbar2 = fig.colorbar(im2, cax=cax, orientation='horizontal' )
for label in cbar2.ax.get_xticklabels():
    label.set_rotation(45)
    
axs[1].set_title("SST Neutral \n(May-Jun-Jul 2018)", fontsize=8)
im2 = axs[1].contourf(hum_lon_neutro,  hum_lat_neutro, humedad_neutro, vmin=hm, vmax=hM, levels=hlevels, cmap=rain, zorder=0)
divider = make_axes_locatable(axs[1])
cax = divider.append_axes('bottom', size='5%', pad=0.4, axes_class=maxes.Axes)
cbar2 = fig.colorbar(im2, cax=cax, orientation='horizontal' )
for label in cbar2.ax.get_xticklabels():
    label.set_rotation(45)

axs[2].set_title("SST Anomalia MIN \n(Oct-Nov-Dic 2020)", fontsize=8)
im2 = axs[2].contourf(hum_lon_anomin,  hum_lat_anomin, humedad_anomin, vmin=hm, vmax=hM, levels=hlevels, cmap=rain, zorder=0)
divider = make_axes_locatable(axs[2])
cax = divider.append_axes('bottom', size='5%', pad=0.4, axes_class=maxes.Axes)
cbar2 = fig.colorbar(im2,  cax=cax, orientation='horizontal' )
for label in cbar2.ax.get_xticklabels():
    label.set_rotation(45)

for i in range(3):
    axs[i].add_feature(COASTLINE, linewidth=0.5)

    gl = axs[i].gridlines(crs=PlateCarree(), draw_labels=True, linestyle='--', zorder=0)
    gl.yformatter = LATITUDE_FORMATTER
    gl.xformatter = LONGITUDE_FORMATTER
    gl.right_labels = False
    gl.top_labels = False

fig.tight_layout()
plt.show()

print("Figura 2: Anomalias sobre el pacifico. Imagen superior anomalía máxima, la del medio anomalía neutra e imagen inferior anomalía mínima. En las 3, recuadrado se encuentra la región 3.4.")